In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


In [ ]:
df = spark.read.format("csv").option("header","true").load("Files/products/products.csv")
# df now is a Spark DataFrame containing CSV data from "Files/products/products.csv".
display(df)

In [3]:
df.write.format('delta').saveAsTable('managed_products')

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, 5, Finished, Available)

In [4]:
df.write.format('delta').saveAsTable('external_products', path='abfss://DzungFabricWs@onelake.dfs.fabric.microsoft.com/DzungLH.Lakehouse/Files/external_products')

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, 6, Finished, Available)

In [5]:
%%sql

DESCRIBE FORMATTED managed_products;

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, 7, Finished, Available)

<Spark SQL result set with 14 rows and 3 fields>

In [6]:
%%sql  

DESCRIBE FORMATTED external_products;

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, 8, Finished, Available)

<Spark SQL result set with 15 rows and 3 fields>

In [7]:
%%sql

drop table managed_products;
DROP TABLE external_products;

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
%%sql

CREATE TABLE products
USING DELTA
LOCATION 'Files/external_products';

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, 12, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [10]:
%%sql

SELECT * FROM products;

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, 13, Finished, Available)

<Spark SQL result set with 295 rows and 4 fields>

In [11]:
%%sql

UPDATE products
SET ListPrice = ListPrice * 0.9
WHERE Category = 'Mountain Bikes';

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, 14, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [12]:
%%sql

DESCRIBE HISTORY products;

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, 15, Finished, Available)

<Spark SQL result set with 2 rows and 15 fields>

In [13]:
delta_tabe_path = 'Files/external_products'

current_data = spark.read.format('delta').load(delta_tabe_path)
display(current_data)

original_data = spark.read.format('delta').option('versionAsOf', 0).load(delta_tabe_path)
display(original_data)

StatementMeta(, 1571840f-ec7f-4af8-9c43-d8cdedaf56b9, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, a5d5f265-849c-471f-b478-4d8ff1706fe3)

SynapseWidget(Synapse.DataFrame, 79b7df5e-a7d1-437f-911f-2c5f3702f457)

In [2]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

#create a folder for stream data
input_path = 'Files/data/'
mssparkutils.fs.mkdirs(input_path)

#create schema for stream data and start streaming
jsonSchema = StructType([
    StructField('device', StringType(), False),
    StructField('status', StringType(), False)
])

iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(input_path)

#input some event data to input folder
device_data = '''{"device":"Dev1", "status":"ok"}
{"device":"Dev1", "status":"ok"}
{"device":"Dev1", "status":"ok"}
{"device":"Dev2", "status":"error"}
{"device":"Dev1", "status":"ok"}
{"device":"Dev1", "status":"error"}
{"device":"Dev2", "status":"ok"}
{"device":"Dev2", "status":"error"}
{"device":"Dev1", "status":"ok"}
'''

mssparkutils.fs.put(input_path + 'data.txt', device_data, True)
print('Source stream created ...')

StatementMeta(, c82edc47-bb67-4212-9e40-1da86fec33f6, 4, Finished, Available)

Source stream created ...


In [6]:
#write stream data to delta table
delta_stream_tbl_path = 'Tables/iotdevicedata'
checkpoint_path = 'Files/delta/checkpoint'

deltastream = iotstream.writeStream.format('delta').option('checkpointLocation', checkpoint_path).start(delta_stream_tbl_path)
print('Streaming to delta sink ...')

StatementMeta(, c82edc47-bb67-4212-9e40-1da86fec33f6, 8, Finished, Available)

Streaming to delta sink ...


In [10]:
%%sql

SELECT * FROM iotdevicedata;

StatementMeta(, c82edc47-bb67-4212-9e40-1da86fec33f6, 12, Finished, Available)

<Spark SQL result set with 1000 rows and 2 fields>

In [5]:
df = spark.read.text("Files/data/data.txt")
# df now is a Spark DataFrame containing text data from "Files/data/data.txt".
display(df)

StatementMeta(, c82edc47-bb67-4212-9e40-1da86fec33f6, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1371fa24-85d1-4a68-b80b-06bf98f0f6b8)

In [9]:
# Add more data to the source stream
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(input_path + "more-data.txt", more_data, True)

StatementMeta(, c82edc47-bb67-4212-9e40-1da86fec33f6, 11, Finished, Available)

True

In [11]:
deltastream.stop()

StatementMeta(, c82edc47-bb67-4212-9e40-1da86fec33f6, 13, Finished, Available)